In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import datetime

from typing import List, Dict

Matplotlib is building the font cache; this may take a moment.


# Stock Prices EDA

## Compute Sharp Ratio

Risk-free rate is ~ $0.08$ for 2021

$s_a=\frac{E[R_a - R_b]}{\sigma_a}$


In [24]:
class SmigStock:

    DATE_FORMAT = "%Y-%m-%d"
    RISK_FREE_RATE = 0.0006

    @property
    def tickers(self):
        return self._tickers

    @tickers.setter
    def tickers(self, tickers: List[str]):
        if tickers and isinstance(tickers, list):
            self._tickers = tickers

    def add_ticker(self, ticker: str) -> None:
        self.tickers.append(ticker)

    @property
    def start_date(self) -> "datetime.datetime":
        return self._start_date

    @start_date.setter
    def start_date(self, value: str) -> None:
        if not isinstance(value, str):
            return
        try:
            self._start_date = datetime.datetime.strptime(value, SmigStock.DATE_FORMAT)
        except Exception as e:
            print(str(e))
            raise

    @property
    def end_date(self) -> "datetime.datetime":
        return self._end_date

    @end_date.setter
    def end_date(self, value: str) -> None:
        if not isinstance(value, str):
            return
        try:
            self._end_date = datetime.datetime.strptime(value, SmigStock.DATE_FORMAT)
        except Exception as e:
            print(str(e))
            raise

    def __init__(self, tickers: List[str], start_date: str, end_date: str) -> None:
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.data = {}

    def get_prices(self) -> None:
        #? Should you redownload price data for all tickers or just ones not already in self.data?
        for ticker in self.tickers:
            self.data[ticker] = {}
            if ticker in self.data:
                print(f"{ticker} data has already been downloaded")
                # continue
            print(f"Downloading stock price data for {ticker}")
            df = web.DataReader(ticker, 'yahoo', self.start_date, self.end_date)
            df.reset_index(inplace=True)
            df = df[['Date', 'Adj Close']]
            df.columns = ['ds', 'y']
            df.set_index('ds', inplace=True)
            df = self.compute_metrics(df)
            print("Download complete")
            self.data[ticker]['prices'] = df
            self.data[ticker]['sharp_ratio'] = self.compute_sharp_ratio(ticker)

    def compute_metrics(self, df):
        df['daily_ret'] = df.diff(axis=0)
        df['daily_percent'] = df['daily_ret'] / df['y'].shift()
        df['excess'] = df.daily_percent-SmigStock.RISK_FREE_RATE
        return df
        
    def compute_sharp_ratio(self, ticker):
        # grab the data
        df = self.data[ticker]['prices']
        s = (df.shape[0]/np.sqrt(252))*df.daily_percent.mean()/df.daily_percent.std()
        print("Sharp Ratio for %s = %s" % (ticker, s))
        return s

In [25]:
tickers = ['TSLA','AAPL','AMZN']
ss = SmigStock(tickers=tickers,start_date='2020-10-01', end_date='2021-02-12')

In [27]:
ss.get_prices()

TSLA data has already been downloaded
Download complete
Sharp Ratio for TSLA = 1.1582631251741264
AAPL data has already been downloaded
Download complete
Sharp Ratio for AAPL = 0.5131143920651774
AMZN data has already been downloaded
Download complete
Sharp Ratio for AMZN = 0.11308695485187312
